In [1]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
import os

from dotenv import load_dotenv

load_dotenv()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

# help functions

In [2]:
## embedding model
def get_embedding_model(model_name="/teamspace/studios/this_studio/bge-small-en-v1.5"):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    return embed_model

# generator model
def get_llm(model_name="llama3-8b-8192"):
    llm = Groq(model=model_name, api_key=os.getenv("GROQ_API"), temperature=0.8)
    return llm

## get deeplake vector database
def get_vector_database(id, dataset_name):
    my_activeloop_org_id = id # "hunter"
    my_activeloop_dataset_name = dataset_name # "Vietnamese-law-RAG"
    dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=False,
    )
    return vector_store

def get_index(vector_store):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    return index

In [3]:
## query generation / rewriting
from llama_index.core import PromptTemplate

query_str = "Vượt đèn đỏ sẽ bị gì?"

prompt_for_generating_query = PromptTemplate(
    """Bạn là một trợ lý xuất sắc trong việc tạo ra các câu truy vấn tìm kiếm liên quan. Dựa trên câu truy vấn đầu vào dưới đây, hãy tạo ra {num_queries} truy vấn tìm kiếm liên quan, mỗi câu trên một dòng. Lưu ý, trả lời bằng tiếng Việt và chỉ trả về các truy vấn đã tạo ra.

### Câu truy vấn đầu vào: {query}

### Các câu truy vấn:"""
)

def generate_queries(llm, query_str, num_queries=4):
    fmt_prompt = prompt_for_generating_query.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries =  response.text.split("\n")
    return queries

def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.retrieve(query))
    
    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, tasks)):
        results_dict[(query, i)] = query_result
    
    return results_dict

In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

def get_bm25_retriever(index, similarity_top_k=13625):
    source_nodes = index.as_retriever(similarity_top_k=similarity_top_k).retrieve("test")
    nodes = [x.node for x in source_nodes]
    bm25 = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=3)
    return bm25

# naive RAG

- from vectorstor -> retrieve relevant nodes
- feed those node into LLMs to generate questions

In [5]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

embed_model = get_embedding_model()
Settings.embed_model = embed_model

llm = get_llm()
Settings.llm = llm
vector_store = get_vector_database("hunter", "Vietnamese-law-RAG")
index = get_index(vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = get_bm25_retriever(index)

Deep Lake Dataset in hub://hunter/Vietnamese-law-RAG already exists, loading from the storage


In [14]:
from llama_index.llms.ollama import Ollama

vistral = Ollama(model="phogpt", request_timeout=120.0)
Settings.llm = get_llm()

In [6]:
## create retriever and query engine
retriever = index.as_retriever(similarity_top_k=3)
query_engine = index.as_query_engine(similarity_top_k=3)

In [40]:
query_engine.get_prompts()

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7f73d7aad630>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

In [8]:
query = "How to be fluent foreign language faster?"
print(query_engine.query(query))

Immerse yourself in the language by listening to native speakers, practicing speaking and writing, and engaging with native speakers.


### evaluate

In [9]:
import pandas as pd

df = pd.read_json("qa_data.json")
df.head()

,question,answer,context
0,"Câu hỏi: Theo điều 22, trong bao nhiêu ngày ph...",Câu trả lời: Phải công khai trong thời hạn ba ...,Node ID: node_9417\nText: 5. Thời điểm công kh...
1,Câu hỏi:,- Bạn biết được khổ đường sắt là gì không?,Node ID: node_12463\nText: 7. Công trình công ...
2,"Câu hỏi: Theo quy định, ai có thẩm quyền quyết...",Câu trả lời: Người có thẩm quyền quyết định di...,Node ID: node_9876\nText: Vị trí mới của trạm ...
3,"Câu hỏi: Theo luật, ai được công nhận có quyền...",Câu trả lời: Luật cho phép đối tượng bao gồm đ...,Node ID: node_9287\nText: Người có quyền tự mì...
4,Câu hỏi: Bộ Lao động - Thương binh và Xã hội c...,Câu trả lời: Doanh nghiệp dịch vụ được miễn ph...,Node ID: node_5243\nText: 2. Trong thời hạn 05...


In [10]:
from llama_index.evaluation import RetrieverEvaluator

# We can evaluate the retievers with different top_k values.
for i in [2, 4, 6, 8, 10]:
    retriever = vector_index.as_retriever(similarity_top_k=i)
    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )
    eval_results = await retriever_evaluator.aevaluate_dataset(qc_dataset)
    print(display_results_retriever(f"Retriever top_{i}", eval_results))

ModuleNotFoundError: No module named 'llama_index.evaluation'

In [11]:
from llama_index.core.evaluation import RetrieverEvaluator

metrics = ["mrr", "hit_rate"]

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=retriever
)

In [24]:
print(df.iloc[27, 2])
print(df.iloc[27, 0])
print(df.iloc[27, 1])

Node ID: node_7480
Text: Điều 25. Trách nhiệm của cơ quan, tổ chức và người dân Thủ đô
1. Hội đồng nhân dân,  Ủy ban nhân dân, Chủ tịch Ủy ban nhân dân các
cấp chính quyền của  thành phố Hà Nội trong phạm vi nhiệm vụ, quyền
hạn của mình thực hiện các quy định của pháp  luật về Thủ đô và chịu
trách nhiệm về những vi phạm, yếu kém xảy ra trong công tác xây dựng,
phát t...
Score:  0.599

Câu hỏi: Theo pháp luật về Thủ đô Hà Nội, thì ai có trách nhiệm giám sát việc thi hành Luật Thủ đô?
Câu trả lời: Hội đồng nhân dân thành phố Hà Nội.


In [25]:
# try it out on a sample query
sample_query = df.iloc[27, 0]
sample_expected = embed_model.get_text_embedding(df.iloc[27, 2])

eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(df.iloc[27, 2])
print(eval_result)

Node ID: node_7480
Text: Điều 25. Trách nhiệm của cơ quan, tổ chức và người dân Thủ đô
1. Hội đồng nhân dân,  Ủy ban nhân dân, Chủ tịch Ủy ban nhân dân các
cấp chính quyền của  thành phố Hà Nội trong phạm vi nhiệm vụ, quyền
hạn của mình thực hiện các quy định của pháp  luật về Thủ đô và chịu
trách nhiệm về những vi phạm, yếu kém xảy ra trong công tác xây dựng,
phát t...
Score:  0.599

Query: Câu hỏi: Theo pháp luật về Thủ đô Hà Nội, thì ai có trách nhiệm giám sát việc thi hành Luật Thủ đô?
Metrics: {'mrr': 0.0, 'hit_rate': 0.0}



# RAGAS

In [27]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.metrics.critique import harmfulness

ImportError: cannot import name 'PydanticOutputParser' from 'langchain_core.output_parsers' (/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/output_parsers/__init__.py)